In [161]:
import numpy as np
import json
import matplotlib.pyplot as plt
import requests 
import tqdm
import tarfile
from clint.textui import progress
import os
from pathlib import Path
import pandas as pd
import scipy.io
import json
import shutil

In [84]:
import matplotlib.pyplot as plt
import cv2

In [13]:
def download_url(url, save_path, chunk_size=128):
    r = requests.get(url, stream=True)
    
    with open(save_path, 'wb') as fd:
        total_length = int(r.headers.get('content-length'))
        
        for chunk in progress.bar(r.iter_content(chunk_size=chunk_size),
                                  expected_size=(total_length/1024) + 1):
            fd.write(chunk)
            fd.flush()
                                  
    return

def extract_file(fname,path='.'):
    if fname.endswith("tar.gz"):
        tar = tarfile.open(fname, "r:gz")
        tar.extractall(path=path)
        tar.close()
    elif fname.endswith("tar"):
        tar = tarfile.open(fname, "r:")
        tar.extractall(path=path)
        tar.close()
    
    return


In [6]:
# Download files

img_url = "http://host.robots.ox.ac.uk/pascal/VOC/voc2010/VOCtrainval_03-May-2010.tar"
annot_url = "http://roozbehm.info/pascal-parts/trainval.tar.gz"

save_path_img = "\\img_zip"
save_path_annot = "\\annot_zip"

curr_path = os.getcwd()
path = Path(curr_path)

save_loc = os.path.join(path.parent,'datasets\\PASCAL-VOC'+save_path_img)
os.mkdir(save_loc)
download_url(img_url,os.path.join(save_loc,'images.tar'))

save_loc = os.path.join(path.parent,'datasets\\PASCAL-VOC'+save_path_annot)
os.mkdir(save_loc)
download_url(annot_url,os.path.join(save_loc,'annotations.tar.gz'))



In [14]:
save_loc = os.path.join(path.parent,'datasets\\PASCAL-VOC'+save_path_annot)
extract_file(save_loc+'\\annotations.tar.gz',path = save_loc)

In [15]:
save_loc = os.path.join(path.parent,'datasets\\PASCAL-VOC'+save_path_img)
extract_file(save_loc+'\\images.tar',path = save_loc)

In [42]:
class_list = os.listdir(save_loc+'\\VOCdevkit\\VOC2010\\ImageSets\\Main')

In [43]:
class_list = [file.split('_')[0] for file in class_list]
class_list = list(filter(lambda x: ('val' not in x)and('train' not in x),
                         class_list))
class_set = set(class_list)
class_set

In [176]:
anno_destination = os.path.join(path.parent,'datasets\\PASCAL-VOC\\xybb-objects\\')
anno_source = os.path.join(path.parent,('datasets\\PASCAL-VOC'+
                                        save_path_annot+
                                        '\\Annotations_Part\\'))
img_source = os.path.join(path.parent,('datasets\\PASCAL-VOC'+
                                       save_path_img+
                                       '\\VOCdevkit\\VOC2010\\JPEGImages\\'))

for ob_class in class_set:
    print(ob_class)
                                       
    new_dir = anno_destination+ob_class
    if not os.path.exists(new_dir):
        os.mkdir(new_dir)
    
    if not os.path.exists(new_dir+'\\bbox'):
        os.mkdir(new_dir+'\\bbox')
        
    if not os.path.exists(new_dir+'\\mask'):
        os.mkdir(new_dir+'\\mask')
        
    if not os.path.exists(new_dir+'\\rgb'):
        os.mkdir(new_dir+'\\rgb')
    
    
    img_list = pd.read_csv((save_loc+'\\VOCdevkit\\VOC2010\\ImageSets\\Main\\'
                           +ob_class+'_trainval.txt'),
                          header=None,index_col=None,sep='\t')
                                       
    
    img_list = img_list[0].str.split(' ',expand=True)
    img_list.columns = ['img','flag','drop']
    img_list.drop(['drop'],axis=1,inplace=True)
                          
    for image in img_list.img[:5]:
        mat = scipy.io.loadmat(anno_source+image+'.mat')
        img_file = (img_source+image+'.jpg')
        img_dest = os.path.join(path.parent,'datasets\\PASCAL-VOC\\scene\\'+image+'.jpg')
        print(img_file)
        if not os.path.exists(img_dest):
            shutil.move(img_file,img_dest)
        label_list = []
        annotation_dict = {}

        for annotation in mat['anno'][0][0][1][0]:

            print('main',annotation[0])
            label = annotation[0][0]
            annotation_dict[label] = {}
            annotation_dict[label]['mask'] = annotation[2].tolist()
            annotation_dict[label]['bbox'] = np.append(np.min(np.where(annotation[2]==1),
                                                                       axis=1),
                                                       np.max(np.where(annotation[2]==1),
                                                                       axis=1)).tolist()
            annotation_dict[label]['parts'] = {}
            parts_dict = annotation_dict[label]['parts'] 


            if (len(annotation[3])>0)and (len(annotation[3][0])>0):
                for part in annotation[3][0]:
                    print(part[0])
                    part_label = part[0][0]
                    parts_dict[part_label] = {}
                    parts_dict[part_label]['mask'] = part[1].tolist()
                    parts_dict[part_label]['bbox'] = np.append(np.min(np.where(part[1]==1),
                                                                              axis=1),
                                                           np.max(np.where(part[1]==1),
                                                                  axis=1)).tolist()


        with open(anno_destination+ob_class+'\\bbox\\'+image+'.json', 'w') as fp:
            json.dump(annotation_dict, fp)




diningtable
C:\Users\user\Documents\Workspace\New folder\meronymnet\dataset_files\datasets\PASCAL-VOC\img_zip\VOCdevkit\VOC2010\JPEGImages\2008_000002.jpg
main ['tvmonitor']
['screen']
C:\Users\user\Documents\Workspace\New folder\meronymnet\dataset_files\datasets\PASCAL-VOC\img_zip\VOCdevkit\VOC2010\JPEGImages\2008_000003.jpg
main ['train']
['coach_1']
main ['person']
['torso']
['luleg']
['rlleg']
['ruleg']
C:\Users\user\Documents\Workspace\New folder\meronymnet\dataset_files\datasets\PASCAL-VOC\img_zip\VOCdevkit\VOC2010\JPEGImages\2008_000007.jpg
main ['boat']
C:\Users\user\Documents\Workspace\New folder\meronymnet\dataset_files\datasets\PASCAL-VOC\img_zip\VOCdevkit\VOC2010\JPEGImages\2008_000008.jpg
main ['person']
['head']
['lear']
['leye']
['lebrow']
['mouth']
['hair']
['nose']
['torso']
['neck']
['llarm']
['luarm']
['llleg']
['luleg']
['ruleg']
main ['horse']
['head']
['lear']
['rear']
['muzzle']
['torso']
['neck']
['lflleg']
['lfuleg']
['lfho']
['lblleg']
['lbuleg']
['lbho']
['ta

['rfuleg']
['lblleg']
['lbuleg']
['rblleg']
main ['cow']
['torso']
['lflleg']
['lfuleg']
['lblleg']
['lbuleg']
['rblleg']
['rbuleg']
['tail']
sheep
C:\Users\user\Documents\Workspace\New folder\meronymnet\dataset_files\datasets\PASCAL-VOC\img_zip\VOCdevkit\VOC2010\JPEGImages\2008_000002.jpg
main ['tvmonitor']
['screen']
C:\Users\user\Documents\Workspace\New folder\meronymnet\dataset_files\datasets\PASCAL-VOC\img_zip\VOCdevkit\VOC2010\JPEGImages\2008_000003.jpg
main ['train']
['coach_1']
main ['person']
['torso']
['luleg']
['rlleg']
['ruleg']
C:\Users\user\Documents\Workspace\New folder\meronymnet\dataset_files\datasets\PASCAL-VOC\img_zip\VOCdevkit\VOC2010\JPEGImages\2008_000007.jpg
main ['boat']
C:\Users\user\Documents\Workspace\New folder\meronymnet\dataset_files\datasets\PASCAL-VOC\img_zip\VOCdevkit\VOC2010\JPEGImages\2008_000008.jpg
main ['person']
['head']
['lear']
['leye']
['lebrow']
['mouth']
['hair']
['nose']
['torso']
['neck']
['llarm']
['luarm']
['llleg']
['luleg']
['ruleg']
ma

C:\Users\user\Documents\Workspace\New folder\meronymnet\dataset_files\datasets\PASCAL-VOC\img_zip\VOCdevkit\VOC2010\JPEGImages\2008_000009.jpg
main ['cow']
['head']
['lear']
['leye']
['muzzle']
['torso']
['neck']
['lflleg']
['lfuleg']
['rflleg']
['rfuleg']
['lblleg']
['lbuleg']
['rblleg']
main ['cow']
['torso']
['lflleg']
['lfuleg']
['lblleg']
['lbuleg']
['rblleg']
['rbuleg']
['tail']
motorbike
C:\Users\user\Documents\Workspace\New folder\meronymnet\dataset_files\datasets\PASCAL-VOC\img_zip\VOCdevkit\VOC2010\JPEGImages\2008_000002.jpg
main ['tvmonitor']
['screen']
C:\Users\user\Documents\Workspace\New folder\meronymnet\dataset_files\datasets\PASCAL-VOC\img_zip\VOCdevkit\VOC2010\JPEGImages\2008_000003.jpg
main ['train']
['coach_1']
main ['person']
['torso']
['luleg']
['rlleg']
['ruleg']
C:\Users\user\Documents\Workspace\New folder\meronymnet\dataset_files\datasets\PASCAL-VOC\img_zip\VOCdevkit\VOC2010\JPEGImages\2008_000007.jpg
main ['boat']
C:\Users\user\Documents\Workspace\New folder\m

main ['horse']
['head']
['lear']
['rear']
['muzzle']
['torso']
['neck']
['lflleg']
['lfuleg']
['lfho']
['lblleg']
['lbuleg']
['lbho']
['tail']
C:\Users\user\Documents\Workspace\New folder\meronymnet\dataset_files\datasets\PASCAL-VOC\img_zip\VOCdevkit\VOC2010\JPEGImages\2008_000009.jpg
main ['cow']
['head']
['lear']
['leye']
['muzzle']
['torso']
['neck']
['lflleg']
['lfuleg']
['rflleg']
['rfuleg']
['lblleg']
['lbuleg']
['rblleg']
main ['cow']
['torso']
['lflleg']
['lfuleg']
['lblleg']
['lbuleg']
['rblleg']
['rbuleg']
['tail']
bottle
C:\Users\user\Documents\Workspace\New folder\meronymnet\dataset_files\datasets\PASCAL-VOC\img_zip\VOCdevkit\VOC2010\JPEGImages\2008_000002.jpg
main ['tvmonitor']
['screen']
C:\Users\user\Documents\Workspace\New folder\meronymnet\dataset_files\datasets\PASCAL-VOC\img_zip\VOCdevkit\VOC2010\JPEGImages\2008_000003.jpg
main ['train']
['coach_1']
main ['person']
['torso']
['luleg']
['rlleg']
['ruleg']
C:\Users\user\Documents\Workspace\New folder\meronymnet\dataset